In [1]:
!pip install selenium pandas webdriver-manager

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [3]:
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [4]:
def get_reviews(product_url):
    driver.get(product_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'reviewTitle')))
    
    reviews = []
    ratings = []
    
    review_elements = driver.find_elements(By.CLASS_NAME, 'reviewTitle')
    rating_elements = driver.find_elements(By.CLASS_NAME, 'fivestarWidgetStatic-vote')

    for review, rating in zip(review_elements, rating_elements):
        review_text = review.text
        stars = rating.find_elements(By.CLASS_NAME, 'on')
        rating_score = len(stars)  
        
        reviews.append(review_text)
        ratings.append(rating_score)

    return reviews, ratings

In [5]:
all_reviews = []
all_ratings = []

home_page_url = 'https://irecommend.ru/catalog/list/937'
driver.get(home_page_url)

for page in range(70): 
    products = driver.find_elements(By.CLASS_NAME, 'ProductTizer')

    wait = WebDriverWait(driver, 10)

for product in products:
    try:
        title_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'title')))
        product_link = title_element.find_element(By.TAG_NAME, 'a').get_attribute('href')
        reviews, ratings = get_reviews(product_link)
        all_reviews.extend(reviews)
        all_ratings.extend(ratings)
        print(all_reviews)
        print(f"PAGE! {page}")
    except StaleElementReferenceException:
        
        print("Error!")

        
    
    driver.get(home_page_url)
    next_page_button = driver.find_element(By.XPATH, '//a[@title="На страницу номер {}"]'.format(page + 2))
    next_page_button.click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ProductTizer')))

driver.quit()


formatted_reviews = [f'"{review}"' for review in all_reviews]
formatted_ratings = [f'{rating:.1f}' for rating in all_ratings]

df = pd.DataFrame({'text': formatted_reviews, 'rating': formatted_ratings})
df.to_csv('reviews_and_ratings.csv', index=False)

print("Данные выгружены")

Сбор отзывов завершен. Данные сохранены в reviews_and_ratings.csv.
